In [2]:
pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.4 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

In [3]:
df_test = pd.read_csv("test.csv",index_col=0)

In [4]:
#trainフォルダ内のcsvファイルを全て取得
files = glob.glob("train/*.csv")

data_list = []
#filesに格納された４７個のファイルをリストに入れる
for file in files:
    data_list.append(pd.read_csv(file,index_col=0))
#追加したリストデータの結合
df = pd.concat(data_list)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
def data_pre(df):
    nonnull_list = []

    #カラムを呼び出しnull数を計算、０のものをnonnull_listに追加する
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
            
    df = df.drop(nonnull_list,axis=1) 

    df = df.drop(["市区町村名","種類"],axis=1) 

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)
    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上",2000).astype(float)
    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 34 - num
        if "令和" in i: 
            num = float(i.split("令和")[1].split("年")[0])
            year = 4 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 97 - num
        y_list[i] = year
    y_list["戦前"] = 77
    df["建築年"] = df["建築年"].replace(y_list)
    df["取引時点"]
    year = {
        "年第1四半期":"0.25",
        "年第2四半期":"0.5",
        "年第3四半期":"0.75",
        "年第4四半期":"0.99"
    }

    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep

    df["取引時点"]=df["取引時点"].replace(year_list).astype(float)
    
    for i in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
        df[i] = df[i].astype("category")
    
    return df

In [6]:
df = data_pre(df)

In [7]:
df_test = data_pre(df_test)

In [8]:
#外部データを取り込む
land_price = pd.read_csv("land-price-announcement.csv",encoding = "shift-jis")

In [9]:
#省略されないように設定
pd.set_option('display.max_columns', 100)
#データの確認
land_price.head()

,経度,緯度,所在地コード,用途,連番,年次,前年所在地コード,前年用途,前年連番,市区町村名,所在並びに地番,住居表示,行政,地積,利用の現況,利用状況表示,利用区分,建物構造,施設,形状区分,間口（比率）,奥行（比率）,階層（地上）,階層（地下）,前面道路区分,前面道路の方位区分,前面道路の幅員,前面道路の駅前区分,前面道路の舗装状況,側道区分,側道方位区分,交通施設との近接区分,周辺の土地の利用の現況,駅名,駅距離,用途区分,防火区分,都市計画区分,高度地区,森林区分,公園区分,建蔽率,容積率,割増容積率,共通地点区分,対前年変動率,選定年次ビット,Ｓ５８価格,Ｓ５９価格,Ｓ６０価格,...,Ｈ２４価格,Ｈ２５価格,Ｈ２６価格,Ｈ２７価格,Ｈ２８価格,Ｈ２９価格,Ｈ３０価格,Ｈ３１価格,Ｒ２価格,Ｒ３価格,Ｒ４価格,属性移動Ｓ５９,属性移動Ｓ６０,属性移動Ｓ６１,属性移動Ｓ６２,属性移動Ｓ６３,属性移動Ｈ１,属性移動Ｈ２,属性移動Ｈ３,属性移動Ｈ４,属性移動Ｈ５,属性移動Ｈ６,属性移動Ｈ７,属性移動Ｈ８,属性移動Ｈ９,属性移動Ｈ１０,属性移動Ｈ１１,属性移動Ｈ１２,属性移動Ｈ１３,属性移動Ｈ１４,属性移動Ｈ１５,属性移動Ｈ１６,属性移動Ｈ１７,属性移動Ｈ１８,属性移動Ｈ１９,属性移動Ｈ２０,属性移動Ｈ２１,属性移動Ｈ２２,属性移動Ｈ２３,属性移動Ｈ２４,属性移動Ｈ２５,属性移動Ｈ２６,属性移動Ｈ２７,属性移動Ｈ２８,属性移動Ｈ２９,属性移動Ｈ３０,属性移動Ｈ３１,属性移動Ｒ２,属性移動Ｒ３,属性移動Ｒ４
0,508730.562,154999.616,1101,0,1,2022,1101,0,1,札幌中央,北海道 札幌市中央区大通西２８丁目２０３番１０,大通西２８−２−５,1101,495,10000000000000000,_,1,RC,111,台形,10,20,2,1,市道,東,109,_,_,_,_,_,大規模住宅、領事館等が存する閑静な住宅地域,円山公園,200,1中専,_,市街化,0,_,_,60,200,False,False,3.4,1111111111111111111111111111111111111111,126000,133000,140000,...,195000,200000,213000,227000,263000,287000,310000,340000,373000,384000,397000,10001000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,11000000000000,10000000000000,10000000000000,10000010000000,10000001000011,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000
1,508821.829,154950.512,1101,0,2,2022,1101,0,2,札幌中央,北海道 札幌市中央区南１３条西１３丁目９４０番１２,南１３条西１３−２−１０,1101,162,10000000000000000,_,1,W,111,_,10,15,2,1,市道,南,54,_,_,_,_,_,一般住宅、アパート等が建ち並ぶ既成住宅地域,西１１丁目,2100,1住居,_,市街化,0,_,_,60,200,False,False,8.3,0000000000000011111111111111111111111111,0,0,0,...,99000,99000,104000,109000,121000,128000,133000,139000,151000,157000,170000,0,0,0,0,0,0,0,0,0,0,0,0,0,40000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000010000010,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000
2,508840.405,154969.668,1101,0,3,2022,1101,0,3,札幌中央,北海道 札幌市中央区南１０条西９丁目１０４１番２５,南１０条西９−１−４８,1101,172,10000000000000100,共同住宅,1,W,111,_,10,20,2,1,市道,北,54,_,_,_,_,_,アパート、マンション等が密集する住宅地域,中島公園,1000,1住居,_,市街化,0,_,_,60,200,False,False,8.1,1111111111111111111111111111111111111111,110000,116000,122000,...,98000,98000,101000,103000,115000,120000,125000,131000,143000,148000,160000,10001000000000,10000000000000,10000000000000,10000000000000,11000000000000,10000000000000,10000000000000,10010000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000001000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000010,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000
3,508857.995,154930.896,1101,0,4,2022,1101,0,4,札幌中央,北海道 札幌市中央区南１８条西８丁目５９３番１８外,南１８条西８−１−４３,1101,146,10000000000000000,_,1,W,111,_,10,20,2,1,市道,北,109,_,_,_,_,_,一般住宅、マンション等が建ち並ぶ既成住宅地域,幌平橋,800,1住居,_,市街化,0,_,_,60,200,False,True,8.8,0000000000000011111111111111111111111111,0,0,0,...,99000,99000,103000,107000,119000,129000,135000,144000,162000,171000,186000,0,0,0,0,0,0,0,0,0,0,0,0,0,40000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000010,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,100

In [10]:
#駅名、建物構造でgroupby
ex_data = land_price.groupby("駅名").agg({"Ｒ２価格":"mean"}).reset_index()

ex_data = ex_data.rename(columns={"駅名":"最寄駅：名称"})

In [11]:
#df,df_testを結合のためのtarget設定
df["target"] = 0
df_test["target"] = 1

#pd.concatでdf_all
df_all = pd.concat([df,df_test],axis = 0)

In [12]:
#pd.mergeでdf_allとex_dataを合体
df_all_1 = pd.merge(df_all,ex_data,how="left")

In [13]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728492 entries, 43046834 to 47007891
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       728492 non-null  int64   
 1   都道府県名         728492 non-null  category
 2   地区名           727823 non-null  object  
 3   最寄駅：名称        725763 non-null  object  
 4   最寄駅：距離（分）     705337 non-null  float64 
 5   間取り           702227 non-null  object  
 6   面積（㎡）         728492 non-null  float64 
 7   建築年           707341 non-null  float64 
 8   建物の構造         708985 non-null  object  
 9   用途            656026 non-null  object  
 10  今後の利用目的       362246 non-null  category
 11  都市計画          708578 non-null  object  
 12  建ぺい率（％）       704231 non-null  float64 
 13  容積率（％）        704231 non-null  float64 
 14  取引時点          728492 non-null  float64 
 15  改装            659593 non-null  category
 16  取引の事情等        19003 non-null   object  
 17  取引価格（総額）_log  707485

In [14]:
enc_dic = {}
for i, e in enumerate(sorted(list(set(df_all['取引時点'].values)))):
    enc_dic[e] = i
df_all['取引時点_enc'] = df_all['取引時点'].map(enc_dic)

In [15]:
te_dic = {}
time_col = '取引時点_enc'
group_col = '都道府県名'
TARGET = "取引価格（総額）_log"

for i in set(df_all[time_col].values):
    tmp_df = df_all[df_all[time_col] < i]
    te_dic[i] = tmp_df.groupby(group_col)[TARGET].agg('mean').to_dict()

te_dic[50]

{'三重県': 7.099989772836446,
 '京都府': 7.155935848555294,
 '佐賀県': 7.009937258263502,
 '兵庫県': 7.1577428704833395,
 '北海道': 6.9659677936779225,
 '千葉県': 7.155825324965087,
 '和歌山県': 7.022690210815764,
 '埼玉県': 7.169744271396741,
 '大分県': 6.911537826573127,
 '大阪府': 7.158352973821392,
 '奈良県': 7.05688461023469,
 '宮城県': 7.039544253805021,
 '宮崎県': 6.958678653711217,
 '富山県': 7.046920071386995,
 '山口県': 7.045342730349819,
 '山形県': 7.087990602814887,
 '山梨県': 6.8488398886867,
 '岐阜県': 7.0780959134864325,
 '岡山県': 7.038090734927503,
 '岩手県': 6.925002679060589,
 '島根県': 7.1653762147003075,
 '広島県': 7.093449021655071,
 '徳島県': 6.912051981629739,
 '愛媛県': 6.966433011596682,
 '愛知県': 7.107301969238703,
 '新潟県': 6.913835156235078,
 '東京都': 7.3779292046375025,
 '栃木県': 6.950636958071025,
 '沖縄県': 7.160046604428329,
 '滋賀県': 7.166026958168431,
 '熊本県': 6.97018225184329,
 '石川県': 6.929363348877193,
 '神奈川県': 7.258118267209689,
 '福井県': 6.979575815840197,
 '福岡県': 6.992264905042241,
 '福島県': 6.974380579140173,
 '秋田県': 6.941313747112618

In [16]:
def calc_te(row):
    if row[time_col] in te_dic and row[group_col] in te_dic[row[time_col]]:
        return te_dic[row[time_col]][row[group_col]]
    else:
        return 0

df_all[group_col+'_te'] = df_all.apply(calc_te, axis=1)
df_all.head(2)

,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,target,取引時点_enc,都道府県名_te
ID,,,,,,,,,,,,,,,,,,,,,
43046834,43101,熊本県,水前寺,水前寺,7.0,３ＬＤＫ,55.0,36.0,ＲＣ,住宅,NaN,第２種中高層住居専用地域,60.0,200.0,20130.50,未改装,NaN,6.897627,0,31,6.956949
43008716,43105,熊本県,徳王,崇城大学前,45.0,３ＬＤＫ,65.0,30.0,ＲＣ,住宅,住宅,第２種住居地域,60.0,200.0,20200.25,未改装,NaN,7.000000,0,58,6.988382


In [17]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728492 entries, 43046834 to 47007891
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       728492 non-null  int64   
 1   都道府県名         728492 non-null  category
 2   地区名           727823 non-null  object  
 3   最寄駅：名称        725763 non-null  object  
 4   最寄駅：距離（分）     705337 non-null  float64 
 5   間取り           702227 non-null  object  
 6   面積（㎡）         728492 non-null  float64 
 7   建築年           707341 non-null  float64 
 8   建物の構造         708985 non-null  object  
 9   用途            656026 non-null  object  
 10  今後の利用目的       362246 non-null  category
 11  都市計画          708578 non-null  object  
 12  建ぺい率（％）       704231 non-null  float64 
 13  容積率（％）        704231 non-null  float64 
 14  取引時点          728492 non-null  float64 
 15  改装            659593 non-null  category
 16  取引の事情等        19003 non-null   object  
 17  取引価格（総額）_log  707485

In [18]:
df_all["最寄駅：距離（分）_mean"] = df_all["最寄駅：距離（分）"].mean()
df_all["最寄駅：距離（分）_std"] = df_all["最寄駅：距離（分）"].std()
df_all["最寄駅：距離（分）_max-min"] = max(df_all["最寄駅：距離（分）"]) - min(df_all["最寄駅：距離（分）"])

df_all["面積（㎡）_mean"] = df_all["面積（㎡）"].mean()
df_all["面積（㎡）_std"] = df_all["面積（㎡）"].std()
df_all["面積（㎡）_max-min"] = max(df_all["面積（㎡）"]) - min(df_all["面積（㎡）"])

df_all["建築年_mean"] = df_all["建築年"].mean()
df_all["建築年_std"] = df_all["建築年"].std()
df_all["建築年_max-min"] = max(df_all["建築年"]) - min(df_all["建築年"])

df_all["取引時点_mean"] = df_all["取引時点"].mean()
df_all["取引時点_std"] = df_all["取引時点"].std()
df_all["取引時点_max-min"] = max(df_all["取引時点"]) - min(df_all["建築年"])

In [19]:
#model学習時にエラーが出たため再度カテゴリに変換
for i in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
    df_all[i] = df_all[i].astype("category")

In [20]:
#df_allをdf_2,df_test_2に分割
df_2 = df_all.query("target == 0")
df_test_2 = df_all.query("target == 1")

In [21]:
#trainデータをX,yに分割
X = df_2.drop(["取引価格（総額）_log","target"],axis=1)
y = df_2["取引価格（総額）_log"]

In [22]:
#kfold
from sklearn.model_selection import KFold

FOLD = 5
NUM_ROUND = 1000
VERBOSE_EVAL = -1
NUM_CLASS = 1

params = {
    "objective": "regression",
    "metrics": "mae",
    "verbose": -1,
    "num_class": NUM_CLASS 
}

valid_scores = []
models = []
kf = KFold(n_splits=FOLD, shuffle=True, random_state=42)

for fold, (train_indices, valid_indices) in enumerate(kf.split(X,y)):
    X_train, X_valid = X.iloc[train_indices,:],X.iloc[valid_indices,:]
    y_train, y_valid = y.iloc[train_indices],y.iloc[valid_indices]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)
    
    model = lgb.train(params,lgb_train,valid_sets=lgb_eval,num_boost_round=NUM_ROUND,early_stopping_rounds=100)
    
    y_valid_pred = model.predict(X_valid)
    score = mae(y_valid, y_valid_pred)
    print(f'fold {fold} log loss: {score}')
    valid_scores.append(score)
    
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score:{cv_score}')

/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's l1: 0.254175
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.237935
[3]	valid_0's l1: 0.223525
[4]	valid_0's l1: 0.210951
[5]	valid_0's l1: 0.199822
[6]	valid_0's l1: 0.189587
[7]	valid_0's l1: 0.180971
[8]	valid_0's l1: 0.173124
[9]	valid_0's l1: 0.165926
[10]	valid_0's l1: 0.159739
[11]	valid_0's l1: 0.154197
[12]	valid_0's l1: 0.149109
[13]	valid_0's l1: 0.145016
[14]	valid_0's l1: 0.140933
[15]	valid_0's l1: 0.137188
[16]	valid_0's l1: 0.133941
[17]	valid_0's l1: 0.131186
[18]	valid_0's l1: 0.128532
[19]	valid_0's l1: 0.126181
[20]	valid_0's l1: 0.123812
[21]	valid_0's l1: 0.121854
[22]	valid_0's l1: 0.1197
[23]	valid_0's l1: 0.117696
[24]	valid_0's l1: 0.115972
[25]	valid_0's l1: 0.114558
[26]	valid_0's l1: 0.113053
[27]	valid_0's l1: 0.111609
[28]	valid_0's l1: 0.110324
[29]	valid_0's l1: 0.109188
[30]	valid_0's l1: 0.107862
[31]	valid_0's l1: 0.106799
[32]	valid_0's l1: 0.105854
[33]	valid_0's l1: 0.104865
[34]	valid_0's l1: 0.1041

/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's l1: 0.253937
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.237507
[3]	valid_0's l1: 0.22336
[4]	valid_0's l1: 0.210639
[5]	valid_0's l1: 0.199582
[6]	valid_0's l1: 0.189388
[7]	valid_0's l1: 0.180603
[8]	valid_0's l1: 0.172965
[9]	valid_0's l1: 0.166063
[10]	valid_0's l1: 0.159831
[11]	valid_0's l1: 0.154219
[12]	valid_0's l1: 0.14917
[13]	valid_0's l1: 0.144933
[14]	valid_0's l1: 0.141075
[15]	valid_0's l1: 0.13747
[16]	valid_0's l1: 0.134283
[17]	valid_0's l1: 0.131273
[18]	valid_0's l1: 0.128621
[19]	valid_0's l1: 0.125975
[20]	valid_0's l1: 0.123899
[21]	valid_0's l1: 0.122035
[22]	valid_0's l1: 0.119958
[23]	valid_0's l1: 0.118155
[24]	valid_0's l1: 0.116438
[25]	valid_0's l1: 0.114681
[26]	valid_0's l1: 0.113241
[27]	valid_0's l1: 0.111842
[28]	valid_0's l1: 0.110563
[29]	valid_0's l1: 0.109383
[30]	valid_0's l1: 0.108203
[31]	valid_0's l1: 0.107226
[32]	valid_0's l1: 0.106361
[33]	valid_0's l1: 0.105457
[34]	valid_0's l1: 0.10413

/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's l1: 0.254148
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.23773
[3]	valid_0's l1: 0.223493
[4]	valid_0's l1: 0.210827
[5]	valid_0's l1: 0.1995
[6]	valid_0's l1: 0.189495
[7]	valid_0's l1: 0.180542
[8]	valid_0's l1: 0.172806
[9]	valid_0's l1: 0.165962
[10]	valid_0's l1: 0.15958
[11]	valid_0's l1: 0.153911
[12]	valid_0's l1: 0.148879
[13]	valid_0's l1: 0.144338
[14]	valid_0's l1: 0.140436
[15]	valid_0's l1: 0.13696
[16]	valid_0's l1: 0.133864
[17]	valid_0's l1: 0.13086
[18]	valid_0's l1: 0.128374
[19]	valid_0's l1: 0.125791
[20]	valid_0's l1: 0.123814
[21]	valid_0's l1: 0.12186
[22]	valid_0's l1: 0.120028
[23]	valid_0's l1: 0.118225
[24]	valid_0's l1: 0.116266
[25]	valid_0's l1: 0.114762
[26]	valid_0's l1: 0.11306
[27]	valid_0's l1: 0.111606
[28]	valid_0's l1: 0.11036
[29]	valid_0's l1: 0.109272
[30]	valid_0's l1: 0.108176
[31]	valid_0's l1: 0.107207
[32]	valid_0's l1: 0.106201
[33]	valid_0's l1: 0.10521
[34]	valid_0's l1: 0.104286
[35]	

/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's l1: 0.254307
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.237775
[3]	valid_0's l1: 0.223535
[4]	valid_0's l1: 0.210863
[5]	valid_0's l1: 0.19971
[6]	valid_0's l1: 0.1898
[7]	valid_0's l1: 0.180571
[8]	valid_0's l1: 0.172762
[9]	valid_0's l1: 0.165684
[10]	valid_0's l1: 0.159397
[11]	valid_0's l1: 0.153843
[12]	valid_0's l1: 0.148863
[13]	valid_0's l1: 0.144316
[14]	valid_0's l1: 0.140279
[15]	valid_0's l1: 0.13691
[16]	valid_0's l1: 0.133695
[17]	valid_0's l1: 0.130768
[18]	valid_0's l1: 0.128169
[19]	valid_0's l1: 0.12578
[20]	valid_0's l1: 0.123591
[21]	valid_0's l1: 0.121452
[22]	valid_0's l1: 0.119736
[23]	valid_0's l1: 0.117908
[24]	valid_0's l1: 0.115979
[25]	valid_0's l1: 0.114236
[26]	valid_0's l1: 0.112755
[27]	valid_0's l1: 0.111431
[28]	valid_0's l1: 0.110197
[29]	valid_0's l1: 0.109103
[30]	valid_0's l1: 0.10802
[31]	valid_0's l1: 0.107013
[32]	valid_0's l1: 0.106074
[33]	valid_0's l1: 0.10525
[34]	valid_0's l1: 0.104346
[3

/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's l1: 0.253011
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.236558
[3]	valid_0's l1: 0.22242
[4]	valid_0's l1: 0.209754
[5]	valid_0's l1: 0.198558
[6]	valid_0's l1: 0.188463
[7]	valid_0's l1: 0.179953
[8]	valid_0's l1: 0.172028
[9]	valid_0's l1: 0.165124
[10]	valid_0's l1: 0.158773
[11]	valid_0's l1: 0.153337
[12]	valid_0's l1: 0.148604
[13]	valid_0's l1: 0.144366
[14]	valid_0's l1: 0.140468
[15]	valid_0's l1: 0.136814
[16]	valid_0's l1: 0.133366
[17]	valid_0's l1: 0.130503
[18]	valid_0's l1: 0.127875
[19]	valid_0's l1: 0.125603
[20]	valid_0's l1: 0.123402
[21]	valid_0's l1: 0.121485
[22]	valid_0's l1: 0.11958
[23]	valid_0's l1: 0.117902
[24]	valid_0's l1: 0.115969
[25]	valid_0's l1: 0.114281
[26]	valid_0's l1: 0.112882
[27]	valid_0's l1: 0.111572
[28]	valid_0's l1: 0.110305
[29]	valid_0's l1: 0.109216
[30]	valid_0's l1: 0.108199
[31]	valid_0's l1: 0.107008
[32]	valid_0's l1: 0.10605
[33]	valid_0's l1: 0.105236
[34]	valid_0's l1: 0.10429

In [23]:
models

In [24]:
df_test_2 = df_test_2.drop(["取引価格（総額）_log","target"],axis=1)

In [25]:
test_pred = pd.DataFrame(np.arange(len(df_test_2)))
test_pred.head()

,0
0,0
1,1
2,2
3,3
4,4


In [26]:
test_pred["first"] = models[0].predict(df_test_2,num_iteration=model.best_iteration)

In [27]:
test_pred["second"] = models[1].predict(df_test_2,num_iteration=model.best_iteration)
test_pred["sard"] = models[2].predict(df_test_2,num_iteration=model.best_iteration)
test_pred["four"] = models[3].predict(df_test_2,num_iteration=model.best_iteration)
test_pred["five"] = models[4].predict(df_test_2,num_iteration=model.best_iteration)

In [28]:
test_pred["pred"] = (test_pred["first"] + test_pred["second"] + test_pred["sard"] + test_pred["four"] + test_pred["five"]) / 5  

In [29]:
df_test["pred"] = test_pred["pred"]
df_test["pred"].to_csv("submit_test3.csv")

In [30]:
test_pred.to_csv("test_pred.csv")